In [ ]:
import os
import numpy as np
import librosa
import tensorflow as tf

def generate_int8_logmel_from_wav(
    wav_path,
    tflite_model_path,
    header_path,
    variable_name="g_logmel_sample_int8"):
    """
    Processes a .wav file to generate int8 quantized Log Mel Energies and
    exports them to a C header file. This is for models that take the
    output of the Mel filterbank as input, before the DCT step.
    """
    try:
        # --- Load the TFLite model to get quantization parameters ---
        if not os.path.exists(tflite_model_path):
            print(f"Error: TFLite model not found at '{tflite_model_path}'")
            return

        interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
        interpreter.allocate_tensors()
        input_details = interpreter.get_input_details()[0]

        # Check if the model input is int8
        if 'quantization' not in input_details or input_details['dtype'] != np.int8:
            print(f"Error: The provided TFLite model's input is not int8 quantized.")
            print(f"Model expects: {input_details['dtype']}")
            return

        # Get the scale and zero point for the input tensor
        input_scale, input_zero_point = input_details['quantization']
        print(f"Model Input Quantization Params: scale={input_scale}, zero_point={input_zero_point}")


        # --- Preprocessing Parameters (Must match your original model) ---
        sample_rate = 16000
        n_fft = 640
        hop_length = 320
        n_mels = 40
        # n_mfcc is no longer needed as we stop before the DCT
        fmin = 20
        fmax = 4000

        # --- Load and Preprocess Audio ---
        y, sr = librosa.load(wav_path, sr=sample_rate)
        if len(y) > sample_rate: y = y[:sample_rate]
        else: y = np.pad(y, (0, max(0, sample_rate - len(y))), 'constant')

        # 1. Calculate Mel Spectrogram
        mel_spec = librosa.feature.melspectrogram(
            y=y, sr=sample_rate, n_fft=n_fft, hop_length=hop_length,
            n_mels=n_mels, fmin=fmin, fmax=fmax, center=False
        )

        # 2. Convert to dB scale (Log Mel Energies)
        db_spec = librosa.power_to_db(mel_spec)

        # Transpose to get (time, features) format -> (49, 40)
        log_mel_features_float = db_spec.T

        # --- Quantize the Float Log Mel Energies to int8 ---
        # Formula: int_val = (float_val / scale) + zero_point
        log_mel_features_int8 = (log_mel_features_float / input_scale) + input_zero_point

        # Clip to the valid int8 range [-128, 127] and cast to int8
        log_mel_features_int8 = np.clip(log_mel_features_int8, -128, 127).astype(np.int8)

        # Flatten the 2D array [49, 40] to a 1D array [1960] for the C header
        log_mel_flat_int8 = log_mel_features_int8.flatten()

        # --- Write to C header file ---
        with open(header_path, 'w') as f:
            f.write(f"/*\n * INT8 Quantized Log Mel Energy data from: {os.path.basename(wav_path)}\n */\n\n")
            f.write("#ifndef LOGMEL_SAMPLE_INT8_H\n")
            f.write("#define LOGMEL_SAMPLE_INT8_H\n\n")
            f.write("#include <stdint.h>\n\n")
            f.write(f"#define {variable_name.upper()}_LEN {len(log_mel_flat_int8)}\n\n")
            f.write(f"const int8_t {variable_name}[{len(log_mel_flat_int8)}] = {{\n    ")

            for i, val in enumerate(log_mel_flat_int8):
                f.write(f"{val}, ")
                if (i + 1) % 16 == 0:
                    f.write("\n    ")

            f.write("\n};\n\n")
            f.write("#endif /* LOGMEL_SAMPLE_INT8_H */\n")

        print(f"Successfully generated int8 Log Mel features into '{header_path}'")
        print(f"Feature shape: {log_mel_features_int8.shape}")
        print(f"Total features: {len(log_mel_flat_int8)}")

    except Exception as e:
        print(f"An error occurred: {e}")

# --- USAGE ---
# 1. Provide the path to the audio file you want to test (e.g., "yes.wav")
input_wav_file = 'yes.wav'

# 2. !! IMPORTANT !! Provide the path to your NEW, SIMPLER quantized .tflite model
quantized_tflite_file = 'model.tflite'

# 3. Define the output header file and C variable name
output_header_file = 'logmel_sample_int8.h'
c_variable_name = 'g_yes_logmel_sample_int8'

if not os.path.exists(input_wav_file):
    print(f"Error: Input WAV file '{input_wav_file}' not found.")
elif not os.path.exists(quantized_tflite_file):
    print(f"Error: TFLite model file '{quantized_tflite_file}' not found.")
else:
    generate_int8_logmel_from_wav(input_wav_file, quantized_tflite_file, output_header_file, c_variable_name)


Model Input Quantization Params: scale=0.1015625, zero_point=-128
Successfully generated int8 Log Mel features into 'logmel_sample_int8.h'
Feature shape: (49, 40)
Total features: 1960
